In [25]:
import json
import os
import base64

def encode_image_to_base64(image_path):
    with open(image_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def coco_to_labelme(coco_file, output_dir):
    # COCO JSON 파일 읽기
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)

    # 출력 디렉토리 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 이미지 별로 처리
    for image in coco_data['images']:
        labelme_data = {
            "version": "4.5.6",
            "flags": {},
            "shapes": [],
            "imagePath": image['file_name'],
            "imageData": None,
            "imageHeight": image['height'],
            "imageWidth": image['width']
        }

        # 현재 이미지에 대한 어노테이션 추가
        for annotation in coco_data['annotations']:
            if annotation['image_id'] == image['id']:
                shape = {
                    "label": coco_data['categories'][annotation['category_id']]['name'],
                    "points": [],
                    "group_id": None,
                    "shape_type": "polygon",
                    "flags": {}
                }
                
                # COCO의 segmentation 정보를 LabelMe 형식으로 변환
                segmentation = annotation['segmentation'][0]
                points = [(int(segmentation[i]), int(segmentation[i + 1])) for i in range(0, len(segmentation), 2)]
                shape['points'] = points
                
                labelme_data['shapes'].append(shape)
                
        image_path = os.path.join("./image", image['file_name'])
        if os.path.exists(image_path):
            labelme_data['imageData'] = encode_image_to_base64(image_path)
        
        # 이미지 별로 LabelMe JSON 파일로 저장
        output_file = os.path.join(output_dir, f"{os.path.splitext(image['file_name'])[0]}.json")
        with open(output_file, 'w') as f:
            json.dump(labelme_data, f, indent=4)

# 사용 예시
coco_to_labelme('./_annotations.coco.json', './')